In [ ]:
# # KSSB 1 1차 chunk

# #KSSB 1
# import fitz
# import re
# from langchain.schema import Document

# # PDF 열기
# pdf_path = r"D:\TECHLAB\2PJCHATBOT\data\raw\2. 공개초안_지속가능성 공시기준서 제1호 '일반사항'.pdf"
# doc = fitz.open(pdf_path)

# # # 5페이지부터 본문 텍스트 추출
# text = "\n".join(doc[i].get_text() for i in range(4, len(doc)))

# def postprocess_text(text):
#     """
#     텍스트 후처리: '제호 1' → '제1호' 등 PDF 깨짐 복원
#     """
#     # '제호 1' 또는 '제 호 1' → '제1호'
#     text = re.sub(r"제\s*호\s*(\d+)", r"제\1호", text)
#     return text

# # 후처리 적용
# full_text = postprocess_text(text)

# from langchain.schema import Document

# # ✅ 큰 제목 목록
# main_sections = [
#     "목적",
#     "적용범위",
#     "개념적 기반",
#     "공정한 표시",
#     "중요성",
#     "보고기업",
#     "연계된 정보",
#     "핵심요소"
#     "일반 요구사항",
#     "지침의 원천",
#     "공시의 위치",
#     "보고시기",
#     "비교정보",
#     "준수 문구",
#     "기후 외 다른 지속가능성 관련 위험 및 기회에 대한 정보",
#     "공시",
#     "불확실성 및 오류",  # 👉 실제 등장하는 키워드
#     "판단",
#     "측정불확실성",
#     "오류",
#     "부록"
# ]

# # ✅ 줄 단위 분리
# lines = full_text.splitlines()
# section_starts = {}

# # ✅ 각 섹션이 들어간 줄의 위치 기록
# for i, line in enumerate(lines):
#     for section in main_sections:
#         if section in line:
#             # '불확실성 및 오류' → '판단, 불확실성 및 오류'로 변경
#             title = "판단, 불확실성 및 오류" if "불확실성 및 오류" in line else section
#             if title not in section_starts:
#                 section_starts[title] = i

# # ✅ 섹션 정렬 및 자르기
# sorted_sections = sorted(section_starts.items(), key=lambda x: x[1])

# section_chunks = []
# for i, (title, start_idx) in enumerate(sorted_sections):
#     end_idx = sorted_sections[i + 1][1] if i + 1 < len(sorted_sections) else len(lines)
#     chunk_text = "\n".join(lines[start_idx:end_idx]).strip()

#     section_chunks.append(Document(
#         page_content=chunk_text,
#         metadata={"chunk_id": f"source_{i+1:02}", "source": f"지속가능성 공시기준서 제1호_{title}", "title": "지속가능성 공시기준서 제1호"}
#     ))

# #KSSB 1
# from langchain.schema import Document

# # 1. 판단 청크에서 부록 시작 위치 파악
# appendix_start_idx = section_chunks[18].page_content.find("부록")

# # 2. 판단과 부록으로 분리
# before_appendix = section_chunks[18].page_content[:appendix_start_idx].strip()
# appendix_from_here = section_chunks[18].page_content[appendix_start_idx:].strip()

# # 3. 새로운 판단, 부록 청크 생성
# revised_section_19 = Document(
#     page_content=before_appendix,
#     metadata={"chunk_id": "source_19", "source": "지속가능성 공시기준서 제1호_오류", "title": "지속가능성 공시기준서 제1호"}
# )

# merged_appendix = Document(
#     page_content=appendix_from_here,
#     metadata={"chunk_id": "source_20", "source": "지속가능성 공시기준서 제1호_부록", "title": "지속가능성 공시기준서 제1호"}
# )

# # 4. 기존 섹션 리스트 수정 (0~4 + 수정된 판단 + 통합 부록)
# section_chunks = section_chunks[:18] + [revised_section_19 , merged_appendix]

# # 5. 부록 줄 단위 분해
# appendix_lines = appendix_from_here.splitlines()

# # 6. 부록 라벨 수동 정의 및 줄 위치 추출
# labels = ["부록 A", "부록 B", "부록 C", "부록 E", "부록 F", "부속지침"]
# appendix_starts = {}
# for label in labels:
#     for i, line in enumerate(appendix_lines):
#         if label in line and label not in appendix_starts:
#             appendix_starts[label] = i
#             break

# # 7. 부록 청크 생성
# sorted_appendices = sorted(appendix_starts.items(), key=lambda x: x[1])

# appendix_chunks = []
# for i, (title, start_idx) in enumerate(sorted_appendices):
#     end_idx = sorted_appendices[i + 1][1] if i + 1 < len(sorted_appendices) else len(appendix_lines)
#     chunk_text = "\n".join(appendix_lines[start_idx:end_idx]).strip()

#     appendix_chunks.append(Document(
#         page_content=chunk_text,
#         metadata={
#             "chunk_id": f"source_{len(section_chunks) + i + 1:02}",
#             "source": f"지속가능성 공시기준서 제1호_{title}",
#         }
#     ))

# # 8. 최종 전체 청크 결합
# all_chunks = section_chunks[:-1] + appendix_chunks  # 기존 section_07 '부록(전체)' 제거


# import fitz
# from langchain.schema import Document

# # 1. PDF 열기
# pdf_path = r"D:\TECHLAB\2PJCHATBOT\data\raw\2. 공개초안_지속가능성 공시기준서 제1호 '일반사항'.pdf"
# doc = fitz.open(pdf_path)


# def postprocess_text(doc):
#     """
#     텍스트 후처리: '제호 1' → '제1호' 등 PDF 깨짐 복원
#     """
#     # '제호 1' 또는 '제 호 1' → '제1호'
#     text = re.sub(r"제\s*호\s*(\d+)", r"제\1호", text)

#     return doc

# # 2. 페이지 구간 정의 (0-indexed 기준)
# appendix_page_ranges = [
#     ("부록 A 용어의 정의", 20, 23),   # 21~23쪽
#     ("부록 B 적용지침", 23, 40),   # 24~40쪽
#     ("부록 C 지침의 원천", 40, 41),   # 41쪽
#     ("부록 D 유용한 지속가능성 관련 재무정보의 질적특성", 41, 49), # 42~49쪽
#     ("부록 E 핵심요소에 대한 공시 요구사항", 49, 61),   # 50~61쪽
#     ("부록 F 시행일 및 경과규정", 61, 62),   # 62쪽
#     # ("부속지침", 62, len(doc))  # 63쪽 ~ 마지막
# ]

# # 3. Document로 분할
# appendix_chunks = []
# start_idx = len(section_chunks)

# for i, (title, start_page, end_page) in enumerate(appendix_page_ranges):
#     text = "\n".join(doc[p].get_text() for p in range(start_page, end_page)).strip()
    
#     appendix_chunks.append(Document(
#         page_content=text,
#         metadata={
#             "chunk_id": f"source_{start_idx + i:02}",
#             "source": f"지속가능성 공시기준서 제1호_{title}",
#             "title": "지속가능성 공시기준서 제1호"
#         }
#     ))

# # section_06 (판단) 이전까지만 유지
# section_chunks = section_chunks[:19]

# # 병합
# all_chunks = section_chunks + appendix_chunks

# # 출력 확인
# # print(f"✅ 총 청크 수: {len(all_chunks)}")
# # for doc in all_chunks:
# #     print(f"📌 {doc.metadata['title']} - {doc.metadata['source']} : {len(doc.page_content)}자")


# # ✅ 부속지침 전체 텍스트에서 '예시지침' 이후 텍스트만 추출
# appendix_text = merged_appendix.page_content
# ig_start = appendix_text.find("지속가능성 공시기준서 제1호\n지속가능성 관련 재무정보 공시를 위한 일반사항\n‘\n’\n예시지침")
# subguideline_text = appendix_text[ig_start:].strip()

# # ✅ 분할 기준 정의 (이미지 기준)
# sub_sections = [
#     "예시지침",
#     "주요이용자",
#     "주요이용자의 정보수요 충족",
#     "공개적으로 이용가능한 정보의 이용",
#     "지침의 원천의 적용",
#     "SASB 기준",
#     "CDSB 프레임워크 적용 지침",
#     "예시사례",
#     "사례 \n단일 사업부문 영위 기업",
#     "사례 \n다양한 활동을 하는 대형 복합기업"
# ]

# # ✅ 위치 탐색
# split_indices = []
# for label in sub_sections:
#     idx = subguideline_text.find(label)
#     if idx != -1:
#         split_indices.append((label, idx))

# # ✅ 실제 분할 수행
# # ✅ 실제 분할 수행
# split_indices.sort(key=lambda x: x[1])
# fine_chunks = []

# # 사전 정의된 보기용 이름 매핑
# display_map = {
#     "사례 \n단일 사업부문 영위 기업": "사례 1 - 단일 사업부문 영위 기업",
#     "사례 \n다양한 활동을 하는 대형 복합기업": "사례 2 - 다양한 활동을 하는 대형 복합기업(conglomerate)"
# }

# for i, (raw_title, start_idx) in enumerate(split_indices):
#     end_idx = split_indices[i+1][1] if i+1 < len(split_indices) else len(subguideline_text)
#     chunk = subguideline_text[start_idx:end_idx].strip()

#     # 표시용 이름이 정의돼 있으면 바꾸고, 아니면 그대로 사용
#     display_title = display_map.get(raw_title, raw_title)

#     fine_chunks.append(Document(
#         page_content=chunk,
#         metadata={
#             "source": f"지속가능성 공시기준서 제1호_부속지침_{display_title}",
#             "title": "지속가능성 공시기준서 제1호",
#             "section": display_title
#         }
#     ))


# KSSB1 = all_chunks + fine_chunks

# # ✅ 출력 확인
# print(f"✅ 총 청크 수: {len(KSSB1)}")
# for idx, doc in enumerate(KSSB1):
#     print(f"{idx:03d}. 📌 {doc.metadata['title']} - {doc.metadata['source']} : {len(doc.page_content)}자")

# # 34개


In [ ]:
#KSSB1
import fitz
import re

# ✅ PDF 열기
pdf_path = r"D:\TECHLAB\2PJCHATBOT\data\raw\2. 공개초안_지속가능성 공시기준서 제1호 '일반사항'.pdf"
doc = fitz.open(pdf_path)

# ✅ 텍스트 후처리 함수
def postprocess_text(text):
    text = re.sub(r"제\s*호\s*(\d+)", r"제\1호", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


# ✅ 섹션 위치 찾기 함수
def get_section_positions(text, section_list, section_target_index={}):
    positions = []
    for section in section_list:
        matches = list(re.finditer(re.escape(section), text))
        if not matches:
            continue
        target_index = section_target_index.get(section, 0)
        match = matches[target_index] if target_index < len(matches) else matches[-1]
        positions.append((match.start(), section))
    return positions


# ✅ 1. 본문 처리 (5~21쪽)
body_text = "\n".join(doc[i].get_text() for i in range(4, 21))
body_text = postprocess_text(body_text)

body_sections = [
    "목적",
    "적용범위",
    "개념적 기반",
    "공정한 표시",
    "중요성",
    "보고기업",
    "연계된 정보",
    "핵심요소",
    "일반 요구사항",
    "지침의 원천",
    "공시의 위치",
    "보고시기",
    "비교정보",
    "준수 문구",
    "판단, 불확실성 및 오류",
    "판단",
    "측정불확실성",
    "오류"
]

body_targets = {
    "개념적 기반" : 1, 
    "중요성" : 1,
    "보고기업" : 1, 
    "판단": 3,
    "오류": 1
}
position1 = get_section_positions(body_text, body_sections, body_targets)

chunks1 = []
for i in range(len(position1)):
    start = position1[i][0]
    heading = position1[i][1]
    end = position1[i+1][0] if i+1 < len(position1) else len(body_text)
    chunk = body_text[start:end].strip()
    chunks1.append({
        "page_content": chunk,
        "metadata": {"source": f"지속가능성 공시기준서 제1호_{heading}", "title": "지속가능성 공시기준서 제1호"}
    })

# ✅ 2. 부록 (21~65쪽)
appendix_map = {
    "부록 A. 용어의 정의": (20, 23),
    "부록 B. 적용지침": (23, 40),
    "부록 C. 지침의 원천": (40, 41),
    "부록 D. 유용한 지속가능성 관련 재무정보의 질적특성": (41, 49),
    "부록 E. 핵심요소에 대한 공시 요구사항": (49, 61),
    "부록 F. 시행일 및 경과규정": (61, 62),
}
chunks2 = []
for title, (start, end) in appendix_map.items():
    chunk = "\n".join([doc[i].get_text() for i in range(start, end)])
    chunk = postprocess_text(chunk)
    chunks2.append({
        "page_content": chunk,
        "metadata": {"source": f"지속가능성 공시기준서 제1호_{title}", "title": "지속가능성 공시기준서 제1호"}
    })

# ✅ 3. 부속지침 (65~82쪽)
guidance_text = "\n".join([doc[i].get_text() for i in range(64, 82)])
guidance_text = postprocess_text(guidance_text)
guidance_sections = [
    "예시지침",
    "주요이용자",
    "주요이용자의 정보수요 충족",
    "공개적으로 이용가능한 정보의 이용",
    "지침의 원천의 적용",
    "기준 SASB IG11",
    "프레임워크 적용지침 CDSB IG25",
    "예시사례",
    "사례 단일 사업부문 영위 기업 1",
    "사례 다양한 활동을 하는 대형 복합기업 2"
]

guidance_targets = {
    "SASB 기준" : 1, 
    "CDSB 프레임워크 적용 지침" : 3,
}
position3 = get_section_positions(guidance_text, guidance_sections, guidance_targets)

heading_map = {
    "기준 SASB IG11": "SASB 기준",
    "프레임워크 적용지침 CDSB IG25": "CDSB 프레임워크 적용지침",
    "사례 단일 사업부문 영위 기업 1": "사례 1 - 단일 사업부문 영위 기업",
    "사례 다양한 활동을 하는 대형 복합기업 2": "사례 2 - 다양한 활동을 하는 대형 복합기업"
}

chunks3 = []
for i in range(len(position3)):
    start = position3[i][0]
    heading = position3[i][1]
    end = position3[i+1][0] if i+1 < len(position3) else len(guidance_text)
    chunk = guidance_text[start:end].strip()

    # ✅ heading 매핑 적용
    source_label = heading_map.get(heading, heading)

    chunks3.append({
        "page_content": chunk,
        "metadata": {
            "source": f"지속가능성 공시기준서 제1호_부속지침_{source_label}",
            "title": "지속가능성 공시기준서 제1호"}})

# ✅ 전체 결합 및 확인
documents1 = chunks1 + chunks2 + chunks3


# 기존 dict → langchain Document 변환
from langchain.schema import Document


documents1 = [
    Document(
        page_content=d["page_content"],
        metadata=d["metadata"]
    ) for d in documents1
]


print(f"✅ 총 청크 수: {len(documents1)}")
for idx, doc in enumerate(documents1):
    print(f"{idx:03d}. 📌 {doc.metadata['title']} - {doc.metadata['source']} : {len(doc.page_content)}자")



# ✅ 2차 청크 분할 함수
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_into_chunks(documents1, max_chars=1000):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=20,
        separators=["\n\n", "\n", ".", " ", ""]
    )

    new_docs = []
    for doc in documents1:
        source_id = doc.metadata.get("source", "unknown")
        chunks = splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            new_docs.append(Document(
                page_content=chunk,
                metadata={
                    "chunk_id": f"{source_id}_{i+1:02}",
                    "source": source_id}))
    
    return new_docs


# ✅ 2차 청크 생성 및 저장
second_level_docs = split_into_chunks(documents1)
print(f"✅ 2차 청크 수: {len(second_level_docs)}")

KSSB_01_1st=documents1
KSSB_01_2nd=second_level_docs

# 33개

✅ 총 청크 수: 33
000. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_목적 : 898자
001. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_적용범위 : 833자
002. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_개념적 기반 : 228자
003. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_공정한 표시 : 797자
004. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_중요성 : 320자
005. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_보고기업 : 66자
006. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_연계된 정보 : 579자
007. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_핵심요소 : 392자
008. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_일반 요구사항 : 7자
009. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_지침의 원천 : 1367자
010. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_공시의 위치 : 474자
011. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_보고시기 : 1071자
012. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_비교정보 : 286자
013. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_준수 문구 : 627자
014. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_판단 : 572자
015. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_측정불확실성 : 1172자
016. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_오류 : 1028자
017. 📌 지속가능성 공시기준서 제1호 - 지속가능성 공시기준서 제1호_부록 A. 용어의 정의 : 1659자
018. 📌 지속가능성 공

In [ ]:
# # KSSB 2 1차 chunk

# #KSSB 2
# import fitz
# from langchain.schema import Document

# # 1. PDF 열기
# pdf_path = r"D:\TECHLAB\2PJCHATBOT\data\raw\3. 공개초안_지속가능성 공시기준서 제2호 ‘기후 관련 공시사항’.pdf"
# doc = fitz.open(pdf_path)

# # 2. 글자 기반 섹션 분할 (본문: 5페이지부터)
# full_text = "\n".join(doc[i].get_text() for i in range(4, len(doc)))

# # ✅ 섹션 제목 리스트
# section_titles = ["목적", "적용범위", "핵심요소", "거버넌스", "전략", "위험관리", "지표 및 목표"]
# split_positions = []

# # ✅ 제목 위치 탐색
# for title in section_titles:
#     pos = full_text.find(title)
#     if pos != -1:
#         split_positions.append((title, pos))

# split_positions.sort(key=lambda x: x[1])

# # ✅ 텍스트 기반 청크 생성
# text_chunks = []
# for i, (title, start) in enumerate(split_positions):
#     end = split_positions[i + 1][1] if i + 1 < len(split_positions) else len(full_text)
#     content = full_text[start:end].strip()
#     text_chunks.append(Document(
#         page_content=content,
#         metadata={"chunk_id": f"source_{i+1:02}", "source": f"지속가능성 공시기준서 제2호_{title}", "title" : "지속가능성 공시기준서 제2호"}
#     ))

# # ✅ 부록 페이지 기준 정의 (0-indexed)
# appendix_page_ranges = [
#     ("부록 A 용어의 정의", 24, 30),   # 25~30쪽
#     ("부록 B 적용지침", 30, 56),   # 31~56쪽
#     ("부록 C 시행일 및 경과규정", 56, 58),   # 57~58쪽
#     # ("부속지침", 58, len(doc))  # 59~마지막
# ]

# # ✅ 부록 페이지 청크 생성
# appendix_chunks = []
# start_idx = len(text_chunks)
# for i, (title, start, end) in enumerate(appendix_page_ranges):
#     text = "\n".join(doc[p].get_text() for p in range(start, end)).strip()
#     appendix_chunks.append(Document(
#         page_content=text,
#         metadata={"chunk_id": f"source_{start_idx + i + 1:02}", "source": f"지속가능성 공시기준서 제2호_{title}", "title" : "지속가능성 공시기준서 제2호"}
#     ))


# sub_sections = [
#     "예시지침",
#     "예시사례",
#     "온실가스 배출량의 통합 및 세분화",
#     "사례 \n회계상 연결실체와 기타 피투자자 간의 스코프",
#     "사례 \n스코프 \n카테고리별 스코프",
#     "사례 \n온실가스 종류별 온실가스 배출량 세분화\n3—\nIE13\n사례 \n및 \n는 기업의 온실가스 절대",
#     "자산운용 부문의 금융배출량 세분화\nIE25\n",
#     "사례 \n적극적\n전략과 소극적\n전략에 따른 세분화\n4",
#     "사례 \n자산군별 세분화\n5"
# ]

# display_map = {
#     "사례 \n회계상 연결실체와 기타 피투자자 간의 스코프":
#         "사례 1—회계상 연결실체와 기타 피투자자 간의 스코프 1(Scope 1) 및 스코프 2(Scope 2) 온실가스 배출량 세분화",
#     "사례 \n스코프 \n카테고리별 스코프":
#         "사례 2—스코프 3(Scope 3) 카테고리별 스코프 3 온실가스 배출량 세분화",
#     "사례 \n온실가스 종류별 온실가스 배출량 세분화\n3—\nIE13\n사례 \n및 \n는 기업의 온실가스 절대":
#         "사례 3—온실가스 종류별 온실가스 배출량 세분화",
#     "자산운용 부문의 금융배출량 세분화\nIE25\n":
#         "자산운용 부문의 금융배출량 세분화",
#     "사례 \n적극적\n전략과 소극적\n전략에 따른 세분화\n4":
#         "사례 4—적극적(active) 전략과 소극적(passive) 전략에 따른 세분화",
#     "사례 \n자산군별 세분화\n5":
#         "사례 5—자산군별 세분화"
# }


# # 1. 61쪽 이후 텍스트 추출
# sub_text = "\n".join(doc[p].get_text() for p in range(60, len(doc)))

# # 2. 기준 텍스트로 위치 찾기
# split_indices = []
# for label in sub_sections:
#     idx = sub_text.find(label)
#     if idx != -1:
#         split_indices.append((label, idx))

# # 3. 분할
# split_indices.sort(key=lambda x: x[1])
# fine_chunks = []
# for i, (raw_title, start_idx) in enumerate(split_indices):
#     end_idx = split_indices[i+1][1] if i+1 < len(split_indices) else len(sub_text)
#     chunk = sub_text[start_idx:end_idx].strip()

#     display_title = display_map.get(raw_title, raw_title)

#     fine_chunks.append(Document(
#         page_content=chunk,
#         metadata={
#             "chunk_id": f"source_{i+1:02}",
#             "source": f"지속가능성 공시기준서 제2호_부속지침_{display_title}",
#             "title": "지속가능성 공시기준서 제2호"
#         }
#     ))



# # ✅ 전체 청크 병합
# all_chunks = text_chunks + appendix_chunks + fine_chunks



# # ✅ 출력 확인
# print(f"✅ 총 청크 수: {len(all_chunks)}")
# for idx, doc in enumerate(all_chunks):
#     print(f"{idx:03d}. 📌 {doc.metadata['title']} - {doc.metadata['source']} : {len(doc.page_content)}자")


# # 19개


# # ✅ 2차 청크 분할 함수
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.schema import Document

# def split_into_chunks(all_chunks, max_chars=1000):
#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=max_chars,
#         chunk_overlap=20,
#         separators=["\n\n", "\n", ".", " ", ""]
#     )

#     new_docs = []
#     for doc in all_chunks:
#         source_id = doc.metadata.get("source", "unknown")
#         chunks = splitter.split_text(doc.page_content)
#         for i, chunk in enumerate(chunks):
#             new_docs.append(Document(
#                 page_content=chunk,
#                 metadata={
#                     "chunk_id": f"{source_id}_{i+1:02}",
#                     "source": source_id,
#                 }
#             ))
    
#     return new_docs


# # ✅ 2차 청크 생성 및 저장
# second_level_docs = split_into_chunks(all_chunks)
# print(f"✅ 2차 청크 수: {len(second_level_docs)}")

# KSSB_02_1st=all_chunks
# KSSB_02_2nd=second_level_docs


In [ ]:
# # KSSB 3 1차 chunk

# #KSSB 3
# import fitz
# import re
# from langchain.schema import Document

# # 1. PDF 로드
# pdf_path = r"D:\TECHLAB\2PJCHATBOT\data\raw\4. 공개초안_지속가능성 공시기준서 제101호 '정책 목적을 고려한 추가 공시사항'.pdf"
# doc = fitz.open(pdf_path)

# # 2. 글자 기반 분할 섹션 제목
# text_sections = ["목적", "적용범위", "개념적 기반", "공정한 표시", "보고실체", "일반 요구사항", "공시의 위치", "보고시기", "비교정보", "공시 요구사항", "국내 법률이나 규정에 따라 공개 중인 정보", "정부 부처의 정책 목적을 달성하기 위한 정보", "시행일", "경과규정"]
# text = "\n".join(doc[i].get_text() for i in range(3, len(doc)))

# # ✅ 텍스트 줄 단위 분할 및 정확한 제목 기준 청크 분할
# lines = text.splitlines()
# section_chunks = []
# current_title = None
# current_content = []

# def save_chunk(title, content):
#     if title and content:
#         section_chunks.append(Document(
#             page_content="\n".join(content).strip(),
#             metadata={"chunk_id": f"source_{len(section_chunks)+1:02}","source": f"지속가능성 공시기준서 제101호_{title}", "title" : "지속가능성 공시기준서 제101호"}
#         ))

# for line in lines:
#     line_clean = line.strip()
#     if line_clean in text_sections:
#         save_chunk(current_title, current_content)
#         current_title = line_clean
#         current_content = []
#     else:
#         current_content.append(line)

# # 마지막 청크 저장
# save_chunk(current_title, current_content)



# # ✅ 페이지 기준 섹션 정의 (0-indexed 기준)
# page_sections = [
#     ("부록 A", 14, len(doc)),
# ]

# # ✅ 페이지 기반 섹션 추가
# for i, (title, start, end) in enumerate(page_sections):
#     text = "\n".join(doc[p].get_text() for p in range(start, end)).strip()

#     section_chunks.append(Document(
#         page_content=text,
#         metadata={"chunk_id": f"source_{len(section_chunks) + 1:02}", "source": f"지속가능성 공시기준서 제101호_{title}", "title" : "지속가능성 공시기준서 제101호"}
#     ))




# # ✅ 출력 확인
# print(f"✅ 총 청크 수: {len(section_chunks)}")
# for idx, doc in enumerate(all_chunks):
#     print(f"{idx:03d}. 📌 {doc.metadata['title']} - {doc.metadata['source']} : {len(doc.page_content)}자")

# # 15개


# # ✅ 2차 청크 분할 함수
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.schema import Document

# def split_into_chunks(all_chunks, max_chars=1000):
#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=max_chars,
#         chunk_overlap=20,
#         separators=["\n\n", "\n", ".", " ", ""]
#     )

#     new_docs = []
#     for doc in all_chunks:
#         source_id = doc.metadata.get("source", "unknown")
#         chunks = splitter.split_text(doc.page_content)
#         for i, chunk in enumerate(chunks):
#             new_docs.append(Document(
#                 page_content=chunk,
#                 metadata={
#                     "chunk_id": f"{source_id}_{i+1:02}",
#                     "source": source_id,
#                 }
#             ))
    
#     return new_docs


# # ✅ 2차 청크 생성 및 저장
# second_level_docs = split_into_chunks(all_chunks)
# print(f"✅ 2차 청크 수: {len(second_level_docs)}")

# KSSB_101_1st=section_chunks
# KSSB_101_2nd=second_level_docs


In [ ]:
# IFRS S1 국문 시도 1

import fitz
import re

# ✅ PDF 열기
pdf_path = r"D:\TECHLAB\2PJCHATBOT\data\raw\2.IFRS_S1_‘지속가능성_관련_재무정보_공시를_위한_일반_요구사항’_번역본_최종안_통합본.pdf"
doc = fitz.open(pdf_path)

# ✅ 텍스트 후처리
def postprocess_text(text):
    text = re.sub(r"제\s*호\s*(\d+)", r"제\1호", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

# ✅ 청크 리스트 초기화
documents = []


def get_section_positions(text: str, section_list: list[str], section_target_index: dict[str, int] = {} )-> list[tuple[int, str]]:
    """
    텍스트 내에서 각 섹션 제목의 등장 위치를 찾아 (시작 위치, 섹션명) 튜플 리스트로 반환.
    - section_list: 찾고자 하는 섹션 제목 리스트
    - section_target_index: 특정 섹션에 대해 몇 번째 등장(index)을 사용할지 명시하는 dict
      (예: {"지표 및 목표": 2} → 세 번째 등장 사용)
    - 지정하지 않은 경우 기본적으로 첫 번째 등장(index 0)을 사용
    """
    positions = []

    for section in section_list:
        matches = list(re.finditer(re.escape(section), text))

        if not matches:
            continue  # 섹션이 존재하지 않으면 skip

        # 섹션별 타겟 등장 위치 (기본값은 0)
        target_index = section_target_index.get(section, 0)

        # 타겟 index가 존재하지 않을 경우 마지막 등장 위치 fallback
        if target_index < len(matches):
            match = matches[target_index]
        else:
            match = matches[-1]

        positions.append((match.start(), section))

    # 등장 순서 기준 정렬
    # positions.sort()
    return positions


# ✅ 본문 (5~29쪽)
body_text = "\n".join([doc[i].get_text() for i in range(4, 29)])
body_text = postprocess_text(body_text)

body_sections = [
    "목적", 
    "적용범위", 
    "개념적 기반", 
    "공정한 표시", 
    "중요성", 
    "보고기업", 
    "연계된 정보",
    "핵심요소",
    "거버넌스", 
    "전략", 
    "위험관리", 
    "지표 및 목표", 
    "일반 요구사항", 
    "지침의 원천", 
    "공시의 위치",
    "보고시기", 
    "비교정보", 
    "준수 문구", 
    "판단, 불확실성 및 오류", 
    "판단", 
    "측정불확실성", 
    "오류"
]

#########여기서 함수 사용 (진행 중)
body_section_targets = {
    "중요성" : 1,
    "보고기업" : 1,
    "거버넌스" : 4,
    "전략" : 4,
    "위험관리" : 2,
    "지표 및 목표" : 2,
    "일반 요구사항" : 3,
    "판단" : 9,
    "측정불확실성" : 1,
    "오류" : 1,
}

position1 = get_section_positions(body_text, body_sections, body_section_targets)


chunks1 = []
for i in range(len(position1)):
    start = position1[i][0]
    heading = position1[i][1]
    end = position1[i+1][0] if i+1 < len(position1) else len(body_text)
    chunk = body_text[start:end].strip()
    chunks1.append({
        "page_content": chunk,
        "metadata": {
            "source": f"IFRS 지속가능성 공시기준 S1_{heading}",
            "title": "IFRS 지속가능성 공시기준 S1"
        }
    })



position2 = []
# ✅ 부록 (30~60쪽)
appendix_map = {
    "부록 A. 용어의 정의": (29, 32),
    "부록 B. 적용지침": (32, 49),
    "부록 C. 지침의 원천": (49, 50),
    "부록 D. 유용한 지속가능성 관련 재무정보의 질적특성": (50, 58),
    "부록 E. 시행일 및 경과규정": (58, 60),
}

chunks2 = []
for title, (start, end) in appendix_map.items():
    chunk = "\n".join([doc[i].get_text() for i in range(start, end)])
    chunk = postprocess_text(chunk)
    chunks2.append({
        "page_content": chunk,
        "metadata": {
            "source": f"IFRS 지속가능성 공시기준 S1_{title}",
            "title": "IFRS 지속가능성 공시기준 S1"
        }
    })

# ✅ 부속지침 (62~80쪽)
guidance_text = "\n".join([doc[i].get_text() for i in range(63, 80)])
guidance_text = postprocess_text(guidance_text)

guidance_sections = [
    "예시지침",
    "주요이용자",
    "주요이용자의 정보수요 충족",
    "공개적으로 이용가능한 정보의 이용",
    "지침의 원천의 적용",
    "SASB 기준",
    "CDSB 프레임워크 적용지침",
    "예시사례",
    "SASB 기준 IE2 사례 1과 2는 기업이",
    "사례 1 – 단일 사업부문 영위 기업",
    "사례 2 – 다양한 활동을 하는 대형 복합기업(conglomerate)"
]


#########여기 수정########
body_section_targets = {
    "예시지침" : 1,
    "SASB 기준" : 1,
    "CDSB 프레임워크 적용지침" : 3,
    # "단어" : 1,
}

position3 = get_section_positions(guidance_text, guidance_sections, body_section_targets)

# chunks3 = []
# for i in range(len(position3)):
#     start = position3[i][0]
#     heading = position3[i][1]
#     end = position3[i+1][0] if i+1 < len(position3) else len(guidance_text)
#     chunk = guidance_text[start:end].strip()
#     chunks3.append({
#         "page_content": chunk,
#         "metadata": {
#             "source": f"IFRS 지속가능성 공시기준 S1_부속지침_{heading}",
#             "title": "IFRS 지속가능성 공시기준 S1"
#         }
#     })

chunks3 = []
for i in range(len(position3)):
    start = position3[i][0]
    heading = position3[i][1]
    end = position3[i+1][0] if i+1 < len(position3) else len(guidance_text)
    chunk = guidance_text[start:end].strip()

    # ✅ 강제 source 이름 지정
    source_label = "SASB 기준" if "SASB 기준 IE2 사례" in heading else heading

    chunks3.append({
        "page_content": chunk,
        "metadata": {
            "source": f"IFRS 지속가능성 공시기준 S1_부속지침_{source_label}",
            "title": "IFRS 지속가능성 공시기준 S1"
        }
    })



# ✅ 결론도출근거 (81~160쪽)
conclusion_text = "\n".join([doc[i].get_text() for i in range(83, len(doc))])
conclusion_text = postprocess_text(conclusion_text)

conclusion_sections = [
    "도입", 
    "개요", 
    "다른 GAAP에 따라 작성된 재무제표", 
    "다른 IFRS 지속가능성 공시기준서와의 관계",
    "비례성(proportionality)", 
    "배경", 
    "글로벌 기준선, 그리고 관할권 및 규제 이니셔티브와의 상호운용성", 
    "목적", 
    "지속가능성 관련 위험 및 기회", 
    "지속가능성 관련 위험 및 기회의 식별", 
    "개념적 기반",
    "공정한 표시", 
    "중요성", 
    "보고기업", 
    "연계된 정보", 
    "핵심요소", 
    "거버넌스", 
    "전략", 
    "위험관리",
    "지표 및 목표",
    "일반 요구사항", 
    "지침의 원천", 
    "공시의 위치", 
    "보고시기", 
    "비교정보", 
    "준수 문구",
    "판단, 불확실성 및 오류", 
    "판단", 
    "측정불확실성", 
    "오류", 
    "시행일", 
    "경과규정"
]

position4 = []

#########여기 수정########
body_section_targets = {
    # "단어" : 1,
    "목적" : 19,
    "지속가능성 관련 위험 및 기회" : 16,
    "지속가능성 관련 위험 및 기회의 식별" : 1,
    "중요성" : 17,
    "보고기업" : 5,
    "연계된 정보" : 1,
    "핵심요소" : 4,
    "거버넌스" : 9,
    "전략" : 14,
    "위험관리" : 7,
    "지표 및 목표" : 7,
    "일반 요구사항" : 10,
    "지침의 원천" : 3, 
    "공시의 위치" : 1, 
    "보고시기" : 2, 
    "비교정보" : 2,
    "준수 문구" : 1,
    "판단" : 34,
    "측정불확실성" : 6, 
    "오류" : 5,
    "경과규정" : 3,
}

position4 = get_section_positions(conclusion_text, conclusion_sections, body_section_targets)

chunks4 = []
for i in range(len(position4)):
    start = position4[i][0]
    heading = position4[i][1]
    end = position4[i+1][0] if i+1 < len(position4) else len(conclusion_text)
    chunk = conclusion_text[start:end].strip()
    chunks4.append({
        "page_content": chunk,
        "metadata": {
            "source": f"IFRS 지속가능성 공시기준 S1_결론도출근거_{heading}",
            "title": "IFRS 지속가능성 공시기준 S1"
        }
    })


documents1 = chunks1 + chunks2 + chunks3 + chunks4



# 기존 dict → langchain Document 변환
from langchain.schema import Document

documents1 = [
    Document(
        page_content=d["page_content"],
        metadata=d["metadata"]
    ) for d in documents1
]



print(f"✅ 총 청크 수: {len(documents1)}")
for idx, doc in enumerate(documents1):
    print(f"{idx:03d}. 📌 {doc.metadata['title']} - {doc.metadata['source']} : {len(doc.page_content)}자")



# ✅ 2차 청크 분할 함수
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_into_chunks(documents1, max_chars=1000):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=20,
        separators=["\n\n", "\n", ".", " ", ""]
    )

    new_docs = []
    for doc in documents1:
        source_id = doc.metadata.get("source", "unknown")
        chunks = splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            new_docs.append(Document(
                page_content=chunk,
                metadata={
                    "chunk_id": f"{source_id}_{i+1:02}",
                    "source": source_id,
                }
            ))
    
    return new_docs


# ✅ 2차 청크 생성 및 저장
second_level_docs = split_into_chunks(documents1)
print(f"✅ 2차 청크 수: {len(second_level_docs)}")

KOR_IFRS_S1_1st=documents1
KOR_IFRS_S1_2nd=second_level_docs

✅ 총 청크 수: 70
000. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_목적 : 861자
001. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_적용범위 : 484자
002. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_개념적 기반 : 226자
003. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_공정한 표시 : 797자
004. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_중요성 : 304자
005. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_보고기업 : 71자
006. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_연계된 정보 : 554자
007. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_핵심요소 : 421자
008. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_거버넌스 : 1206자
009. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_전략 : 3903자
010. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_위험관리 : 834자
011. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_지표 및 목표 : 1572자
012. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_일반 요구사항 : 7자
013. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_지침의 원천 : 1566자
014. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_공시의 위치 : 676자
015. 📌 IFRS 지속가능성 공시기준 S1 - IFRS 지속가능성 공시기준 S1_보고시기 : 877자
016. 📌 IFRS 지속가능성 공시기준 S1 - IFR

In [ ]:
# IFRS S2 시도 1

import fitz
import re

# ✅ PDF 열기
pdf_path = r"D:\TECHLAB\2PJCHATBOT\data\raw\4.IFRS_S2_‘기후_관련_공시’_번역본_최종안_통합본.pdf"
doc = fitz.open(pdf_path)

# ✅ 텍스트 후처리
def postprocess_text(text):
    text = re.sub(r"제\s*호\s*(\d+)", r"제\1호", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

documents = []

def get_section_positions(text: str, section_list: list[str], section_target_index: dict[str, int] = {} )-> list[tuple[int, str]]:
    """
    텍스트 내에서 각 섹션 제목의 등장 위치를 찾아 (시작 위치, 섹션명) 튜플 리스트로 반환.
    - section_list: 찾고자 하는 섹션 제목 리스트
    - section_target_index: 특정 섹션에 대해 몇 번째 등장(index)을 사용할지 명시하는 dict
      (예: {"지표 및 목표": 2} → 세 번째 등장 사용)
    - 지정하지 않은 경우 기본적으로 첫 번째 등장(index 0)을 사용
    """
    positions = []

    for section in section_list:
        matches = list(re.finditer(re.escape(section), text))

        if not matches:
            continue  # 섹션이 존재하지 않으면 skip

        # 섹션별 타겟 등장 위치 (기본값은 0)
        target_index = section_target_index.get(section, 0)

        # 타겟 index가 존재하지 않을 경우 마지막 등장 위치 fallback
        if target_index < len(matches):
            match = matches[target_index]
        else:
            match = matches[-1]

        positions.append((match.start(), section))

    # 등장 순서 기준 정렬
    # positions.sort()
    return positions


#################################################################################

# ✅ 본문 (4~22쪽)
body_text = "\n".join([doc[i].get_text() for i in range(4, 23)])
body_text = postprocess_text(body_text)

body_sections = [
    "목적", 
    "적용범위", 
    "핵심요소", 
    "거버넌스", 
    "전략", 
    "위험관리", 
    "지표 및 목표"
    ]

position1, chunks1 = [], []

#########여기 수정########
body_section_targets = {
    # "단어" : 1,
    "목적" : 1,
    "전략" : 2,
    "위험관리" : 1,
}

position1 = get_section_positions(body_text, body_sections, body_section_targets)

for i in range(len(position1)):
    start = position1[i][0]
    heading = position1[i][1]
    end = position1[i+1][0] if i+1 < len(position1) else len(body_text)
    chunk = body_text[start:end].strip()
    chunks1.append({
        "page_content": chunk,
        "metadata": {
            "source": f"IFRS 지속가능성 공시기준 S2_{heading}",
            "title": "IFRS 지속가능성 공시기준 S2"
        }
    })

# ✅ 부록 (23~56쪽)
appendix_map = {
    "부록 A. 용어의 정의": (23, 27),
    "부록 B. 적용지침": (28, 53),
    "부록 C. 시행일 및 경과규정": (54, 55),
}

position2, chunks2 = [], []
for title, (start, end) in appendix_map.items():
    chunk = "\n".join([doc[i].get_text() for i in range(start, end + 1)])
    chunk = postprocess_text(chunk)
    chunks2.append({
        "page_content": chunk,
        "metadata": {
            "source": f"IFRS 지속가능성 공시기준 S2_{title}",
            "title": "IFRS 지속가능성 공시기준 S2"
        }
    })

# ✅ 부속지침 (58~83쪽)
guidance_text = "\n".join([doc[i].get_text() for i in range(59, 84)])
guidance_text = postprocess_text(guidance_text)

guidance_sections = [
    "예시지침",
    "예시사례",
    "온실가스 배출량의 통합 및 세분화",
    "사례 1 - 회계상 연결실체와 기타 피투자자 간의 스코프 1(Scope 1) 및 스코프 2(Scope 2) 온실가스 배출량 세분화",
    "사례 2—스코프 3(Scope 3) 카테고리별 스코프 3 온실가스 배출량 세분화",
    "사례 3—온실가스 종류별 온실가스 배출량 세분화",
    "자산운용 부문의 금융배출량 세분화",
    "사례 4—적극적(active) 전략과 소극적(passive) 전략에 따른 세분화",
    "사례 5—자산군별 세분화",
    "IFRS S2 이행에 관한 산업기반 지침",
    "도입",
    "구조 및 용어",
    "적용",
    "중요성",
    "적절한 산업(들)의 선택",
    "SASB 기준과의 양립가능성",
    "위험 및 기회의 식별과 공시 준비",
    "산업전반 지표 범주를 충족하기 위한 정보의 준비"
]


position3, chunks3 = [], []
for section in guidance_sections:
    m = re.search(re.escape(section), guidance_text)
    if m:
        position3.append((m.start(), section))
position3.sort()

#########여기 수정########
body_section_targets = {
    "IFRS S2 이행에 관한 산업기반 지침" : 2, 
    "도입" : 2,
    "적용" : 18,
    # "단어" : 1,
}


position3 = get_section_positions(guidance_text, guidance_sections, body_section_targets)


for i in range(len(position3)):
    start = position3[i][0]
    heading = position3[i][1]
    end = position3[i+1][0] if i+1 < len(position3) else len(guidance_text)
    chunk = guidance_text[start:end].strip()
    chunks3.append({
        "page_content": chunk,
        "metadata": {
            "source": f"IFRS 지속가능성 공시기준 S2_부속지침_{heading}",
            "title": "IFRS 지속가능성 공시기준 S2"
        }
    })

# ✅ 결론도출근거 (84~끝)
conclusion_text = "\n".join([doc[i].get_text() for i in range(85, len(doc))])
conclusion_text = postprocess_text(conclusion_text)

conclusion_sections = [
    "도입",
    "개요",
    "다른 IFRS 지속가능성 공시기준과의 관계",
    "중요성",
    "배경",
    "비례성(proportionality)",
    "상호운용성",
    "목적 및 적용범위",
    "기후 관련 위험 및 기회",
    "임팩트(impacts) 및 의존성",
    "핵심요소",
    "거버넌스",
    "전략",
    "위험관리",
    "지표 및 목표",
    "시행일",
    "경과규정",
    "비교정보",
    "온실가스 배출량 측정방법",
    "스코프 3 온실가스 배출량",
    "기타 경과규정 고려사항"
]


position4, chunks4 = [], []
for section in conclusion_sections:
    m = re.search(re.escape(section), conclusion_text)
    if m:
        position4.append((m.start(), section))
position4.sort()

#########여기 수정########
body_section_targets = {
    # "단어" : 1,
    "비례성(proportionality)" : 2,
    "기후 관련 위험 및 기회" : 19,
    "거버넌스" : 2,
    "전략" : 9, 
    "위험관리" : 5,
    "지표 및 목표" : 3, 
    "경과규정" : 2,
    "비교정보" : 2,
    "스코프 3 온실가스 배출량" : 43,
}

position4 = get_section_positions(conclusion_text, conclusion_sections, body_section_targets)


for i in range(len(position4)):
    start = position4[i][0]
    heading = position4[i][1]
    end = position4[i+1][0] if i+1 < len(position4) else len(conclusion_text)
    chunk = conclusion_text[start:end].strip()
    chunks4.append({
        "page_content": chunk,
        "metadata": {
            "source": f"IFRS 지속가능성 공시기준 S2_결론도출근거_{heading}",
            "title": "IFRS 지속가능성 공시기준 S2"
        }
    })

# ✅ 모든 문서 통합
documents2 = chunks1 + chunks2 + chunks3 + chunks4

# 기존 dict → langchain Document 변환
from langchain.schema import Document

documents2 = [
    Document(
        page_content=d["page_content"],
        metadata=d["metadata"]
    ) for d in documents2
]



print(f"✅ 총 청크 수: {len(documents2)}")
for idx, doc in enumerate(documents2):
    print(f"{idx:03d}. 📌 {doc.metadata['title']} - {doc.metadata['source']} : {len(doc.page_content)}자")



# ✅ 2차 청크 분할 함수
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document

def split_into_chunks(documents1, max_chars=1000):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=20,
        separators=["\n\n", "\n", ".", " ", ""]
    )

    new_docs = []
    for doc in documents1:
        source_id = doc.metadata.get("source", "unknown")
        chunks = splitter.split_text(doc.page_content)
        for i, chunk in enumerate(chunks):
            new_docs.append(Document(
                page_content=chunk,
                metadata={
                    "chunk_id": f"{source_id}_{i+1:02}",
                    "source": source_id,
                }
            ))
    
    return new_docs


# ✅ 2차 청크 생성 및 저장
second_level_docs = split_into_chunks(documents2)
print(f"✅ 2차 청크 수: {len(second_level_docs)}")

KOR_IFRS_S2_1st=documents2
KOR_IFRS_S2_2nd=second_level_docs

✅ 총 청크 수: 49
000. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_목적 : 305자
001. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_적용범위 : 236자
002. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_핵심요소 : 4자
003. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_거버넌스 : 1470자
004. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_전략 : 5838자
005. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_위험관리 : 1070자
006. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_지표 및 목표 : 5183자
007. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_부록 A. 용어의 정의 : 3323자
008. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_부록 B. 적용지침 : 21231자
009. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_부록 C. 시행일 및 경과규정 : 834자
010. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_부속지침_예시지침 : 1230자
011. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_부속지침_예시사례 : 350자
012. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_부속지침_온실가스 배출량의 통합 및 세분화 : 389자
013. 📌 IFRS 지속가능성 공시기준 S2 - IFRS 지속가능성 공시기준 S2_부속지침_사례 1 - 회계상 연결실체와 기타 피투자자 간의 스코프 1(Scope 1) 및 스코프 2(Scope 2) 온실가스 배출량 세분화 : 688자
014. 📌 IFRS 지속가능성 공시기준 S2 - 

In [ ]:
# 전체 벡터 저장

from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
import key
import math

# ✅ 필요한 변수
openai_api_key = key.key["OPEN_API_KEY"]
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

def save_faiss_vectorstore(docs, embedding_model, save_path):
    batch_size = 100
    total = len(docs)
    num_batches = math.ceil(total / batch_size)

    vectorstore = None
    for i in range(num_batches):
        start = i * batch_size
        end = min(start + batch_size, total)
        batch = docs[start:end]

        if i == 0:
            vectorstore = FAISS.from_documents(batch, embedding_model)
        else:
            vectorstore.add_documents(batch)
        print(f"✅ 1차 배치 {i+1}/{num_batches} 저장 완료")

    vectorstore.save_local(save_path)
    print(f"✅ 저장 완료: {save_path}")


In [ ]:
# save_faiss_vectorstore(KSSB_01_1st, embedding_model, "vectorstores/KSSB_01_1st")

In [ ]:
save_faiss_vectorstore(KSSB_01_2nd, embedding_model, "vectorstores/KSSB_01_2nd")

✅ 1차 배치 1/1 저장 완료
✅ 저장 완료: vectorstores/KSSB_01_2nd


In [ ]:
# save_faiss_vectorstore(KSSB_02_1st, embedding_model, "vectorstores/KSSB_02_1st")

In [ ]:
# save_faiss_vectorstore(KSSB_02_2nd, embedding_model, "vectorstores/KSSB_02_2nd")

In [ ]:
# save_faiss_vectorstore(KSSB_101_1st, embedding_model, "vectorstores/KSSB_101_1st")

In [ ]:
# save_faiss_vectorstore(KSSB_101_2nd, embedding_model, "vectorstores/KSSB_101_2nd")

In [ ]:
# save_faiss_vectorstore(KOR_IFRS_S1_1st, embedding_model, "vectorstores/KOR_IFRS_S1_1st")

In [ ]:
save_faiss_vectorstore(KOR_IFRS_S1_2nd, embedding_model, "vectorstores/KOR_IFRS_S1_2nd")

✅ 1차 배치 1/2 저장 완료
✅ 1차 배치 2/2 저장 완료
✅ 저장 완료: vectorstores/KOR_IFRS_S1_2nd


In [ ]:
# save_faiss_vectorstore(KOR_IFRS_S2_1st, embedding_model, "vectorstores/KOR_IFRS_S2_1st")

In [ ]:
save_faiss_vectorstore(KOR_IFRS_S2_2nd, embedding_model, "vectorstores/KOR_IFRS_S2_2nd")

✅ 1차 배치 1/2 저장 완료
✅ 1차 배치 2/2 저장 완료
✅ 저장 완료: vectorstores/KOR_IFRS_S2_2nd
